# Better Predictions than the Kalman Filter

In this notebook, an alternative to the Kalman filter is introduced, that offers the following advantages:
* It performs better when predicting the measured output.
* It is simpler and more intuitive to tune.

It is assumed that the reader has basic knowledge of what the Kalman filter is, and what it is used for, as well as the basics of Transfer Functions and State-Space models.
This notebook includes simulations that will increase in complexity as features of the Kalman and alternative filters are introduced.

All the Python code in this notebook is executed directly in the browser using `pyodide`, no data is sent to any server. If simulations take too much time to execute, try running them in a faster computer (wink).

First import all the Python packages that will be required for the simulations:

In [ ]:
import matplotlib_inline
import numpy as np
import scipy.signal as sig
from scipy.linalg import solve_discrete_are as dare
import matplotlib.pyplot as plt
import math
import pyarma as pa
from ipywidgets import interact
from sympy import symbols, simplify, fraction, Poly

A typical second order system is used in the simulations, the continuous time transfer function (in the Laplace domain) is:
$$
\frac{y(s)}{u(s)} = \frac{k {\omega}^2}{s^2 + 2 \xi \omega s + {\omega}^2}
$$
Where $s$ is the Laplace variable, $y$ is the system output, $u$ the input $k$ is the gain, $\xi$ is the [damping factor](https://en.wikipedia.org/wiki/Damping) and $\omega$ is the natural frequency.

To be able to simulate it, it is first defined as a continuous time transfer function and then converted to state-space form using `scipy.signal`'s `tf2ss`, which yields the continuous time state-space model:

$$
y(t) = C_{sys} \cdot x(t) + D_{sys} \cdot u(t)
$$
$$
\dot{x}(t) = A_{sys} \cdot x(t) + B_{sys} \cdot u(t)
$$
The transfer function and state-space equations are just different representations of _the same_ system. We use both, because it is easier to design using transfer functions, but it is easier to compute (simulate) using state-space representations (computers like matrices and probably viceversa).

Then the continuous time state-space model is discretized using `scipy.signal`'s `cont2discrete`, and a sampling period `t_step`.

In [ ]:
# function that accepts 2rd ord params and returns pyarma discrete state space matrices
def second_order_ss_model(k = 41, wn = 0.1, gi = 0.2, t_step = 0.25):
    # create state space model
    wn2 = pow(wn, 2)
    num = [k*wn2]
    den = [1, 2*gi*wn, wn2]
    (num, den) = sig.normalize(num, den)
    sys_ss_c = sig.tf2ss(num, den)
    
    # discrete state space model
    (Ad, Bd, Cd, Dd, _) = sig.cont2discrete(sys_ss_c, t_step)
    return pa.mat(Ad), pa.mat(Bd), pa.mat(Cd), pa.mat(Dd)

A couple of functions are defined to create the vectors used to store the simulation results, and the function that is used to display them using `matplotlib`.

In [ ]:
def sim_vectors(x_ini, t_step, t_len):
    # time, input, state, output vectors
    t = pa.regspace(0, t_step, t_len).t()
    u = pa.ones(pa.size(t))
    x = pa.zeros(x_ini.n_rows, t.n_cols)
    y = pa.zeros(1, t.n_cols)
    # set initial conditions
    x[:,0] = x_ini
    return t, u, x, y

def sim_plot(t, x_sys, y_sys, x_estk = None, y_estk = None, x_estf = None, y_estf = None, t_line = None):
    plot_sys = {'color': 'b', 'alpha': 0.75, 'linewidth': 1.2}
    plot_kal = {'color': 'r', 'alpha': 1, 'linewidth': 1.2}
    plot_alt = {'color': 'y', 'alpha': 1, 'linewidth': 1.2}
    legend = {'bbox_to_anchor': (1.001, 0), 'loc': 'lower left', 'fontsize': "8", 'shadow': True, 'framealpha': 1}
    # measurement
    plt.subplot(3, 1, 1)
    plt.plot(t.t(), y_sys.t(), **plot_sys, label='y_sys (measurement)')
    if y_estk is not None:
        plt.plot(t.t(), y_estk.t(), **plot_kal, label='y_estk (kalman)')
    if y_estf is not None:
        plt.plot(t.t(), y_estf.t(), **plot_alt, label='y_estf (alternative)')
    plt.legend(**legend)
    plt.ylim((0, 120))
    plt.grid(alpha=0.3)
    if t_line is not None:
        plt.axvline(x=t_line, **plot_sys, linestyle="--")
    # state 1
    plt.subplot(3, 1, 2)
    plt.plot(t.t(), x_sys[0,:].t(), **plot_sys, label='x1_sys (state 1)')
    if x_estk is not None:
        plt.plot(t.t(), x_estk[0,:].t(), **plot_kal, label='x1_estk (state 1 kalman)')
    if x_estf is not None:
        plt.plot(t.t(), x_estf[0,:].t(), **plot_alt, label='x1_estf (state 1 alternative)')
    plt.legend(**legend)
    plt.ylim((-40, 60))
    plt.grid(alpha=0.3)
    if t_line is not None:
        plt.axvline(x=t_line, **plot_sys, linestyle="--")
    # state 2
    plt.subplot(3, 1, 3)
    plt.plot(t.t(), x_sys[1,:].t(), **plot_sys, label='x2_sys (state 2)')
    if x_estk is not None:
        plt.plot(t.t(), x_estk[1,:].t(), **plot_kal, label='x2_estk (state 2 kalman)')
    if x_estf is not None:
        plt.plot(t.t(), x_estf[1,:].t(), **plot_alt, label='x2_estf (state 2 alternative)')
    plt.legend(**legend)
    plt.ylim((0, 200))
    plt.grid(alpha=0.3)
    if t_line is not None:
        plt.axvline(x=t_line, **plot_sys, linestyle="--")
    plt.xlabel('t')
    plt.show()

The simulation of the second order system response is just a simple `for` loop evaluating the discrete time state-space equations of the model:
$$
y[n] = C_{dsys} \cdot x[n] + D_{dsys} \cdot u[n]
$$
$$
x[n+1] = A_{dsys} \cdot x[n] + B_{dsys} \cdot u[n]
$$
Where $x$ is the state _vector_ of the model and $n$ is just the index in the corresponding array. Since the model is discretized at a sampling period of `t_step`, then $y[n]$ is the system output at time `n*t_step`, $x[n]$ is the system state at time `n*t_step` and $x[n+1]$ is the system state at time `(n+1)*t_step` .

The $dsys$ subscript in the state-space matrices denotes that they belong to the _discretized system_.

The system's state values at time zero, $x[0]$, are the system's _initial conditions_.

In [ ]:
def sim_second_order(
    k = 41,           # gain
    wn = 0.1,         # natural frequency
    gi = 0.2,         # damping factor
    x1_sys_ini = 1,   # initial condition for state 1
    x2_sys_ini = 150, # initial condition for state 2
):
    # time step, time length
    t_step = 0.25; t_len = 200
    # discrete system state space model
    (Ad_sys, Bd_sys, Cd_sys, Dd_sys) = second_order_ss_model(k, wn, gi, t_step)    
    # time, input, system state and output vectors
    (t, u, x_sys, y_sys) = sim_vectors(pa.mat([x1_sys_ini, x2_sys_ini]).t(), t_step, t_len)
    
    # run simulation
    for n in range(0, u.n_cols):
        # system output equation
        y_sys[:,n] = Cd_sys * x_sys[:,n] + Dd_sys * u[:,n]
        # system state equation
        if n < u.n_cols - 1:
            x_sys[:,n+1] = Ad_sys * x_sys[:,n] + Bd_sys * u[:,n]

    # show simulation results
    sim_plot(t, x_sys, y_sys)

The simulation results are displayed using `ipywidgets` which allows to interactivelly discover how the system output changes with respect to it's parameters `k` ($k$), `wn` ($\omega$) and `gi` ($\xi$), _and_ its initial conditions `x1_sys_ini` and `x2_sys_ini` ($x[0]$).

In [ ]:
interact(
    sim_second_order, 
    k = (1, 100, 10), 
    wn = (0.1, 0.35, 0.05), 
    gi = (0.1, 2, 0.1),
    x1_sys_ini = (0, 10, 1), 
    x2_sys_ini = (100, 200, 5)
)

Note that in the previous simulation, the system's input $u$ is zero for all time. It is left to the reader, as an excercise, to add the possibility of defining a non-zero input to the simulation.

For the purpose of showing the Kalman and alternative filter results, the free response of the system with non-zero initial conditions is enough.

## The Kalman Filter

Assume we have a model of the system of interest. That is, in the second order system example, assume we have _estimates_ of $k$, $\omega$ and $\xi$, which we will write as $\hat{k}$, $\hat{\omega}$ and $\hat{\xi}$ (using the hat to denote _estimate_). How can we estimate the state vector $x$ from measurements of the output $y$? One idea is to simulate the system model in parallel (at the same time) with the real system, and use some algorithm to _align_ the model with the measurements of the system. An algorithm to achieve the alignment of the model and the measurements is the [Luenberger Observer](https://en.wikipedia.org/wiki/State_observer).

We can obtain an estimate of the discrete time state-space matrices $\hat{A}_{dsys}$, $\hat{B}_{dsys}$, $\hat{C}_{dsys}$ and $\hat{D}_{dsys}$ using the same procedure as before (`scipy.signal`'s `tf2ss` and `cont2discrete`). The estimates of $x$, which we write as $\hat{x}$, are obtained with a discrete-time Luenberger which has the form:

$$
\hat{y}[n] = \hat{C}_{dsys} \cdot \hat{x}[n] + \hat{D}_{dsys} \cdot u[n]
$$
$$
\hat{x}[n+1] = \hat{A}_{dsys} \cdot \hat{x}[n] + \hat{B}_{dsys} \cdot u[n] + L \left(  y[n] - \hat{y}[n] \right)
$$

The Luenberger observer has the same form as the system model itself, but with a correction factor in the state equation $L ( y[n] - \hat{y}[n] )$, where $L$ is the _observer gain_. The problem is to choose an $L$ that guarantees that our estmated states $\hat{x}[n]$ converge (align) to the real ones $x[n]$. As a note, the continuous time observer has the exact same structure, but with continuous equations.

It turns out that, if the system model is exact (i.e. $\hat{A}_{dsys} = A_{dsys}$, $\hat{B}_{dsys} = B_{dsys}$ and so on), any choice of $L$ that makes the matrix $\left( A_{dsys} - L \cdot C_{dsys} \right)$ have its [eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors) inside the unit circle (absolute eigenvalues less than $1$) guarantees that $\hat{x}[n]$ will eventually converge (align) to $x[n]$ (in continuous time the eigenvalues should be negative). So there are _infinitely_ many $L$s that could solve our problem, so which one to choose?

This is where Kalman comes in, [the Kalman filter](https://en.wikipedia.org/wiki/Kalman_filter#Underlying_dynamic_system_model) is actually a special case of the Luenberger observer where $L$ is chosen to be the solution if an specific optimization problem. For the Kalman filter, the observer gain $L$ is called $K$ to denote that it is Kalman's specific choice of $L$.

The details about Kalman's optimization problem are omitted here, except for what is relevant for actual implementation. The problem is typically described in an stochastic framework, and assumes disturbances and noises with probablities expressed matematically by two covariance matrices; $Q$ and $R$ which have the size of the state vector $x$ (squared) and the size of the output $y$ (squared) respectivelly. The solution to that optimization problem can be obtained by solving a [Ricatti Equation](https://en.wikipedia.org/wiki/Riccati_equation) (RE) for the continuous time case and [an Algebraic Riccati Equation](https://en.wikipedia.org/wiki/Algebraic_Riccati_equation) (ARE) for the discrete time case. The ARE depends on the model's state-space matrices and the covariance matrices $Q$ and $R$, which have to be chosen somehow.

So we have traded the problem of choosing a matrix $L$ for choosing two matrices $Q$ and $R$, great. And we must also solve the (Algebraic) Riccati Equation. Luckily we do not have to solve that equation ourselves, `scipy.linalg`'s  `solve_discrete_are` (imported as `dare`) can do that for us. We can simplify the choice of $Q$ and $R$ by restricting them to be [diagonal](https://en.wikipedia.org/wiki/Diagonal_matrix). Actually to guarantee that the observer converges, $Q$ and $R$ have to be positive-definite (i.e. each diagonal entry of the matrices must be positive).

Here is a Python function that solves the equation and gives us the Kalman gain for given $Q$ and $R$ diagonal entries. We have `q1` and `q2` because the size of $x$ is $2$ for the second order system, and a scalar `r` because the size of $y$ is $1$.

In [ ]:
def kalman_gain(Ad, Cd, q1 = 0.5, q2 = 0.5, r = 0.5):
    Qd = np.array([[q1, 0], [0, q2]])
    Rd = np.array([[r]])
    Pd = pa.mat(dare(Ad.t(), Cd.t(), Qd, Rd))
    Kd = Pd * Cd.t() * pa.pinv(pa.eye(Cd.n_rows, Cd.n_rows) + Cd * Pd * Cd.t())
    return Kd

Before simulating the Kalman filter, a note about an implementation detail; in practice there is the so called "Prediction-Correction" or "Prediction-Update" form of the Kalman filter. That is just a fancy name for splitting the observer equations into two parts; the part that includes only the model's equations and the part that includes the correction factor. So the previous equations are re-written as:
$$
\hat{y}[n] = \hat{C}_{dsys} \cdot \hat{x}[n] + \hat{D}_{dsys} \cdot u[n]
$$
$$
\hat{x}[n+1] = \hat{A}_{dsys} \cdot \hat{x}[n] + \hat{B}_{dsys} \cdot u[n]
$$
$$
\hat{x}[n+1] = \hat{x}[n+1] + L \left(  y[n] - \hat{y}[n] \right)
$$
One advantage of this from is that it allows to disconnect the simulated model from the measurements at any time by omitting the last equation, efectivelly disabling the observer and relying only on the model output.

Without further ado, here is the function that simulates the Kalman filter running in parallel to the second order system and aligning the state variable estimates $\hat{x}$.

In [ ]:
# simulate kalman filter with given diagonal values for the weight matrices
def sim_kalman(q1 = 0.5, q2 = 0.5, r = 0.5, enable = True):
    # fixed simulation parameters
    k = 41; wn = 0.1; gi = 0.2; t_step = 0.25; t_len = 200
    x1_sys_ini = 1; x2_sys_ini = 150
    
    # discrete system state space model (the "real" system)
    (Ad_sys, Bd_sys, Cd_sys, Dd_sys) = second_order_ss_model(k, wn, gi, t_step)    
    # time, input, system state and output vectors
    (t, u, x_sys, y_sys) = sim_vectors(pa.mat([x1_sys_ini, x2_sys_ini]).t(), t_step, t_len)

    # discrete estimated state space model (the "simulated" system)
    # NOTE : here we assume we know the system perfectly
    Ad_est = Ad_sys; Bd_est = Bd_sys; Cd_est = Cd_sys; Dd_est = Dd_sys
    # observer state and output vectors
    # NOTE : here we assume we do not know the system's initial conditions
    # the initial guess is a factor of the real initial conditions
    xerr_factor = 0.75;
    x1_est_ini = xerr_factor*x1_sys_ini;
    x2_est_ini = xerr_factor*x2_sys_ini
    
    # compute kalman observer gain    
    Kd = kalman_gain(Ad_est, Cd_est, q1, q2, r)
    # observer state and output vectors
    (_, _, x_estk, y_estk) = sim_vectors(pa.mat([x1_est_ini, x2_est_ini]).t(), t_step, t_len)
    
    # run simulation
    for n in range(0, u.n_cols):
        # system output equation (measurement)
        y_sys[:,n] = Cd_sys * x_sys[:,n] + Dd_sys * u[:,n]
        # system state equation
        if n < u.n_cols - 1:
            x_sys[:,n+1] = Ad_sys * x_sys[:,n] + Bd_sys * u[:,n]
        
        # observer output equation
        y_estk[:,n] = Cd_est * x_estk[:,n] + Dd_est * u[:,n]
        # observer state equation
        if n < u.n_cols - 1:
            x_estk[:,n+1] = Ad_est * x_estk[:,n] + Bd_est * u[:,n]
            # correction factor
            if enable:
                y_errk = y_sys[:,n] - y_estk[:,n]
                x_estk[:,n+1] = x_estk[:,n+1] + Kd * y_errk

    # show simulation results
    sim_plot(t, x_sys, y_sys, x_estk, y_estk)

Again, using `ipywidgets` we can discover how the Kalman filter responds to different values of the covarance matrices $Q$ and $R$, and what happens if we disable the correction term in the Kalman filter equations.

In [ ]:
interact(
    sim_kalman, 
    q1 = (0.1, 1.0, 0.1),
    q2 = (0.1, 1.0, 0.1),
    r = (0.1, 1.0, 0.1),
    enable = True
)

Note that the estimated states `x1_estk` and `x1_estk` ($\hat{x}[n]$ in red), quickly converge (align) to the "real" system states `x1_sys` and `x2_sys` ($x[n]$ in blue) respectivelly. It is again left to the reader as an excercise to add the possibility of defining a non-zero input to the simulation.

It would appear that, in this simple simulation, the covariance matrices somehow influence the "aggressiveness" of the Kalman filter, i.e. how fast $\hat{x}[n]$ converges to $x[n]$, but it is not straght forward to relate the matrices values `q1`, `q2` and `r` to typical filter specifications, like the [bandwidth](https://en.wikipedia.org/wiki/Bandwidth_(signal_processing)). Since the covariance matrices $Q$ and $R$ are supposed to be related to disturbance and noise respectivelly, let's add some to the simulation.

## Disturbance and Noise

With disturbance, it is typically meant a model mismatch (or modelling error). After all, models are a useful simplifications of the real systems, which might be too complicated or expensive to model exactly.

Let's add to the simulation two types of simple disturbances:

* Parametric disturbance : let's assume $\hat{k}$ is not $k$ anymore, but a factor of it; $\hat{k} = k_{factor} \cdot k$.
* Additive disturbance : let's assume we failed to model a ramp that is added to the system output, which can be modeled with the line equation $ramp = 0.4 \cdot t$.

Let's also add some random noise to the system output using `numpy`'s `random`.

In [ ]:
# simulate kalman filter under different types of disturbance and noise
def sim_kalman_disturbance(q1 = 0.5, q2 = 0.5, r = 0.5, dist_k_factor = 1.0, dist_ramp_output = False, noise_output = True):
    # fixed simulation parameters
    k = 41; wn = 0.1; gi = 0.2; t_step = 0.25; t_len = 200
    x1_sys_ini = 1; x2_sys_ini = 150
    # discrete system state space model
    # NOTE : here we add parametric disturbance
    (Ad_sys, Bd_sys, Cd_sys, Dd_sys) = second_order_ss_model(dist_k_factor * k, wn, gi, t_step)
    # time, input, system state and output vectors
    (t, u, x_sys, y_sys) = sim_vectors(pa.mat([x1_sys_ini, x2_sys_ini]).t(), t_step, t_len)
    # noise vector
    noise = pa.mat(np.random.default_rng(0).random(u.n_cols))

    # discrete estimated state space model
    (Ad_est, Bd_est, Cd_est, Dd_est) = second_order_ss_model(k, wn, gi, t_step)
    # assume we do not know the system's initial conditions, initial guess is a factor
    xerr_factor = 0.75; x1_est_ini = xerr_factor*x1_sys_ini; x2_est_ini = xerr_factor*x2_sys_ini
    
    # compute kalman observer gain    
    Kd = kalman_gain(Ad_est, Cd_est, q1, q2, r)
    # observer state and output vectors
    (_, _, x_estk, y_estk) = sim_vectors(pa.mat([x1_est_ini, x2_est_ini]).t(), t_step, t_len)
    
    # run simulation
    for n in range(0, u.n_cols):
        # system output equation (measurement)
        y_sys[:,n] = Cd_sys * x_sys[:,n] + Dd_sys * u[:,n]
        # additive disturbance at the output
        if dist_ramp_output:
            y_sys[:,n] = y_sys[:,n] + 0.4 * t[:,n]
        if noise_output:
            y_sys[:,n] = y_sys[:,n] + 8.0 * noise[:,n]
        # system state equation
        if n < u.n_cols - 1:
            x_sys[:,n+1] = Ad_sys * x_sys[:,n] + Bd_sys * u[:,n]
        
        # observer output equation
        y_estk[:,n] = Cd_est * x_estk[:,n] + Dd_est * u[:,n]
        # observer state equation
        if n < u.n_cols - 1:
            x_estk[:,n+1] = Ad_est * x_estk[:,n] + Bd_est * u[:,n]
            # correction factor
            y_errk = y_sys[:,n] - y_estk[:,n]
            x_estk[:,n+1] = x_estk[:,n+1] + Kd * y_errk

    # show simulation results
    sim_plot(t, x_sys, y_sys, x_estk, y_estk)

With `ipywidgets` we can discover how the Kalman filter responds to different values of $k_{factor}$ and how it responds with and without the additive ramp disturbance and noise.

In [ ]:
interact(
    sim_kalman_disturbance, 
    q1 = (0.1,1.0, 0.1),
    q2 = (0.1, 1.0, 0.1),
    r = (0.1, 1.0, 0.1),
    dist_k_factor = (0.1, 2, 0.1),
    dist_ramp_output = False,
    noise_output = True
)

Note that this time the estimated states do not converge to the system states, due to the fact that the model and system do not match anymore, as a consequence of adding disturbances and noise. Still, the Kalman filter is able to give reasonable estimates based on the available measurements. This property of the filter performing correctly under disturbances is called _robustness_, so the Kalman filter is robust. And it is also now clear that cafeful tuning of the covariance matrices parameters `q1`, `q2` and `r`, is necessary to achieve a good balance between speed of convergance and how noisy the estimates are, although it is still unclear how to tune them.

There is another usage of the Kalman filter that is very popular, which is making _predictions_. The idea is that, once the estimated states $\hat{x}[n]$ have converged to the system states $x[n]$ (this can be infered by looking at the output error $\left( y[n] - \hat{y}[n] \right)$), then we can use the estimated states to iterate the model equations, to predict where the system is going to land at some point in the future. Let see how we can implement that in our simulation. 

## Prediction

To simulate making predictions with the Kalman filter, the `predict_from` variable is introduced, to allow defining a moment in time from which we wish to disconnect the model from the measurements and see how good the model-based predictons are.

In [ ]:
# simulate kalman filter under different types of disturbance
def sim_kalman_prediction(q1 = 0.5, q2 = 0.5, r = 0.5, dist_k_factor = 1.0, dist_ramp_output = False, noise_output = True, predict_from = 100):
    # fixed simulation parameters
    k = 41; wn = 0.1; gi = 0.2; t_step = 0.25; t_len = 200
    x1_sys_ini = 1; x2_sys_ini = 150
    # discrete system state space model
    # NOTE : here we add parametric disturbance
    (Ad_sys, Bd_sys, Cd_sys, Dd_sys) = second_order_ss_model(dist_k_factor * k, wn, gi, t_step)
    # time, input, system state and output vectors
    (t, u, x_sys, y_sys) = sim_vectors(pa.mat([x1_sys_ini, x2_sys_ini]).t(), t_step, t_len)
    # noise vector
    noise = pa.mat(np.random.default_rng(0).random(u.n_cols))

    # discrete estimated state space model
    (Ad_est, Bd_est, Cd_est, Dd_est) = second_order_ss_model(k, wn, gi, t_step)
    # assume we do not know the system's initial conditions, initial guess is a factor
    xerr_factor = 0.75; x1_est_ini = xerr_factor*x1_sys_ini; x2_est_ini = xerr_factor*x2_sys_ini
    
    # compute kalman observer gain    
    Kd = kalman_gain(Ad_est, Cd_est, q1, q2, r)
    # observer state and output vectors
    (_, _, x_estk, y_estk) = sim_vectors(pa.mat([x1_est_ini, x2_est_ini]).t(), t_step, t_len)
    
    # run simulation
    for n in range(0, u.n_cols):
        # system output equation (measurement)
        y_sys[:,n] = Cd_sys * x_sys[:,n] + Dd_sys * u[:,n]
        # additive disturbance at the output
        if dist_ramp_output:
            y_sys[:,n] = y_sys[:,n] + 0.4 * t[:,n]
        if noise_output:
            y_sys[:,n] = y_sys[:,n] + 8.0 * noise[:,n]
        # system state equation
        if n < u.n_cols - 1:
            x_sys[:,n+1] = Ad_sys * x_sys[:,n] + Bd_sys * u[:,n]
        
        # observer output equation
        y_estk[:,n] = Cd_est * x_estk[:,n] + Dd_est * u[:,n]
        # observer state equation
        if n < u.n_cols - 1:
            x_estk[:,n+1] = Ad_est * x_estk[:,n] + Bd_est * u[:,n]
            # correction factor
            # NOTE : at time equals t >= predict_from we stop taking measurements into consideration
            if t[0,n] < predict_from:
                y_errk = y_sys[:,n] - y_estk[:,n]
                x_estk[:,n+1] = x_estk[:,n+1] + Kd * y_errk

    # show simulation results
    sim_plot(t, x_sys, y_sys, x_estk, y_estk, None, None, predict_from)

Using the `ipywidgets` slider for the `predict_from` variable, we can discover how the predictions perform depending on the point in time that we wish to start the simulations and what disturbances are present. Try disabling all disturbances and noise and compare the results when they are enabled.

In [ ]:
interact(
    sim_kalman_prediction, 
    q1 = (0.1, 1.0, 0.1),
    q2 = (0.1, 1.0, 0.1),
    r = (0.1, 1.0, 0.1),
    dist_k_factor = (0.1, 2, 0.1),
    dist_ramp_output = False,
    noise_output = True,
    predict_from = (10, 180, 10)
)

Note that when the disturbances and noise are disabled, predictions are perfect. This is because:
1. In the absence of disturbances and noise, the model and system match perfectly.
2. At the point where we start predicting (`predict_from`), the estimated states have already converged to the system states.

If the disturbances and noise are enabled, the Kalman filter tracks the measured output just until the defined `predict_from`, but afterwards the output predictions become useless. This means that the Kalman filter is robust while the correction factor is taken into account, but the predictions of the Kalman filter are not.

Let's see if there is an alternative that can help us solve that problem.

## Better Alternative

Let's recall the Luenberger observer structure, of which the Kalman filter is a special case:

$$
\hat{y}[n] = \hat{C}_{dsys} \cdot \hat{x}[n] + \hat{D}_{dsys} \cdot u[n]
$$
$$
\hat{x}[n+1] = \hat{A}_{dsys} \cdot \hat{x}[n] + \hat{B}_{dsys} \cdot u[n] + L \left(  y[n] - \hat{y}[n] \right)
$$

We have seen so far that the main job of the an observer is keeping the output error $\left(  y[n] - \hat{y}[n] \right)$ small, and we would like to do that under the presence of disturbances and noise. This is starting to sound like a typical _tracking_ [control](https://en.wikipedia.org/wiki/Control_theory) problem. We wish $\hat{y}[n]$ to _track_ $y[n]$ with certain robustness and performance requirements.

So why not change the static gain $L$ of the Luenberger observer for a filter $F(s)$? _After all, why not? Why shouldn't we do it?_ [**It can be shown**](https://folk.ntnu.no/skoge/prost/proceedings/ifac2014/media/files/2032.pdf) that the resulting observer structure is, unsurprisingly, another special Luenberger observer. One that includes the filter states $x_{flt}$ and the filter's state-space matrices  $A_{dflt}$, $B_{dflt}$, $C_{dflt}$ and $D_{dflt}$. 

If we include $\hat{B}_{dsys}$ in the feedback loop of the observer (to avoid having to deal with a multiple inputs in the tracking problem), the observer structure in the prediction-correction form, would be the following:

$$
\hat{y}[n] = \hat{C}_{dsys} \cdot \hat{x}[n] + \hat{D}_{dsys} \cdot u[n]
$$
There are two state equations for the predition step:
$$
x_{flt}[n+1] = A_{dflt} \cdot x_{flt}[n]
$$
$$
\hat{x}[n+1] = \hat{A}_{dsys} \cdot \hat{x}[n] + \hat{B}_{dsys} \cdot u[n] + \left( \hat{B}_{dsys} \cdot C_{dflt} \right) \cdot x_{flt}[n]
$$
And two equations for the correction step:
$$
x_{flt}[n+1] = x_{flt}[n+1] + B_{dflt} \cdot \left( y[n] - \hat{y}[n] \right)
$$
$$
\hat{x}[n+1] = \hat{x}[n+1] + \left( \hat{B}_{dsys} \cdot D_{dflt} \right) \cdot \left( y[n] - \hat{y}[n] \right)
$$

Now there is the problem of selecting the filter $F(s)$, luckily, control theory has left us quite [a few options](https://en.wikipedia.org/wiki/Control_theory#See_also) over the last decades. Some examples:
* [Pole Placement](https://en.wikipedia.org/wiki/Full_state_feedback).
* [Loop Shaping](https://en.wikipedia.org/wiki/H-infinity_loop-shaping).
* [Internal Model Control](https://en.wikipedia.org/wiki/Internal_model_(motor_control)) (IMC).

We could even use a [PID controller](https://pidtuner.com), but for didactic puposes and simplicity we will _cheat_ and design an IMC filter that will use the inverted model dynamics. With "_cheat_" it means that, if the model would be unstable or [non-minimum phase](https://en.wikipedia.org/wiki/Minimum_phase), then inverting the dynamics would [not be a good idea](https://www.youtube.com/watch?v=G9apWx4iaks). We know we can get away with this, because our second order system example is both stable and minimum phase. Otherwise, it would be better to design $F(s)$ using loop shaping techniques.

The working principles of the IMC method are very simple (for example to [tune a PID controller](https://folk.ntnu.no/skoge/publications/2001/tuningpaper_reno/tuningpaper_06nov01.pdf)):
* Obtain a model of the system $G(s)$ (we already have that).
* Specify the desired closed loop dynamics $D(s)$.
* Use [the formula](https://en.wikipedia.org/wiki/Closed-loop_transfer_function) for the closed loop transfer function to calculate $F(s) = \frac{1}{G(s)} \cdot \frac{1}{D^{-1}(s) - 1}$.

So now we must simply define how we want the closed loop observer to behave ($D(s)$), typically as a [low pass filter](https://en.wikipedia.org/wiki/Low-pass_filter) where we can specify a pole $\tau$ that is directly related to the filter's bandwith:

$$
D(s) = \frac{\tau}{s + \tau}
$$

It is left as an exercise to show that this choice of $D(s)$ will lead to a [non-proper](https://en.wikipedia.org/wiki/Proper_transfer_function) transfer function for $F(s)$. Therefore we must add another pole to $D(s)$, at frequencies higher than $\tau$, for example:

$$
D(s) = \frac{\tau}{s + \tau} \cdot \frac{10 \cdot \tau}{s + 10 \cdot \tau}
$$

This choice of $D(s)$ should yield a proper $F(s)$. We could do the algebra, but instead let's `sympy` do it for us:

In [ ]:
# accepts 2rd ord params and desired closed loop pole (tau)
# and returns the filter's pyarma discrete state space matrices
def filter_ss_model(k_val = 41, wn_val = 0.1, gi_val = 0.2, tau_val = 0.2, t_step = 0.25):
    tk_val = 10
    # design filter F based on plant model G and desired dynamics D
    s, k, wn, gi, tau, tk = symbols("s k wn gi tau tk")
    G = (k * wn**2) / (s**2 + 2*gi*wn*s + wn**2)
    D = (tau * (tk*tau)) / ((s+tau)*(s+tk*tau))
    F = (1/G) * (1 / (D**-1 - 1))
    num, den = fraction(simplify(F))
    num = num.subs({k: k_val, wn: wn_val, gi: gi_val, tau: tau_val, tk: tk_val}).as_poly(s).all_coeffs()
    den = den.subs({k: k_val, wn: wn_val, gi: gi_val, tau: tau_val, tk: tk_val}).as_poly(s).all_coeffs()
    num = [ float(x) for x in num ]
    den = [ float(x) for x in den ]
    # create state space model
    (num, den) = sig.normalize(num, den)
    filt_ss_c = sig.tf2ss(num, den)
    # discrete state space model
    (Ad, Bd, Cd, Dd, _) = sig.cont2discrete(filt_ss_c, t_step)
    return pa.mat(Ad), pa.mat(Bd), pa.mat(Cd), pa.mat(Dd)

As an exercise for the reader, verify that the $F(s)$ above contains an [integrator](https://en.wikipedia.org/wiki/Integrator).

Below is the function implementing the simulation that includes the comparison of both Kalman filter and alternative filter.

In [ ]:
# simulate alternative filter under different types of disturbance
def sim_filter_prediction(q1 = 0.5, q2 = 0.5, r = 0.5, dist_k_factor = 1.0, dist_ramp_output = False, noise_output = True, predict_from = 100, tau = 0.2):
    # fixed simulation parameters
    t_step = 0.25; t_len = 200
    k = 41; wn = 0.1; gi = 0.2;
    x1_sys_ini = 1; x2_sys_ini = 150
    # discrete system state space model
    # NOTE : here we add parametric disturbance
    (Ad_sys, Bd_sys, Cd_sys, Dd_sys) = second_order_ss_model(dist_k_factor * k, wn, gi, t_step)
    # time, input, system state and output vectors
    (t, u, x_sys, y_sys) = sim_vectors(pa.mat([x1_sys_ini, x2_sys_ini]).t(), t_step, t_len)
    # noise vector
    noise = pa.mat(np.random.default_rng(0).random(u.n_cols))

    # discrete estimated state space model
    (Ad_est, Bd_est, Cd_est, Dd_est) = second_order_ss_model(k, wn, gi, t_step)
    # assume we do not know the system's initial conditions, initial guess is a factor
    xerr_factor = 0.75; x1_est_ini = xerr_factor*x1_sys_ini; x2_est_ini = xerr_factor*x2_sys_ini

    # compute kalman observer gain    
    Kd = kalman_gain(Ad_est, Cd_est, q1, q2, r)
    # observer state and output vectors (kalman)
    (_, _, x_estk, y_estk) = sim_vectors(pa.mat([x1_est_ini, x2_est_ini]).t(), t_step, t_len)
    
    # compute alternative filter
    (Ad_flt, Bd_flt, Cd_flt, Dd_flt) = filter_ss_model(k, wn, gi, tau, t_step)
    # observer state and output vectors (alternative)
    (_, _, x_estf, y_estf) = sim_vectors(pa.mat([x1_est_ini, x2_est_ini]).t(), t_step, t_len)
    # filter state vector
    (_, _, x_flt, _) = sim_vectors(pa.zeros(Ad_flt.n_rows,1), t_step, t_len)
    
    # run simulation
    for n in range(0, u.n_cols):
        # system output equation (measurement)
        y_sys[:,n] = Cd_sys * x_sys[:,n] + Dd_sys * u[:,n]
        # additive disturbance at the output
        if dist_ramp_output:
            y_sys[:,n] = y_sys[:,n] + 0.4 * t[:,n]
        if noise_output:
            y_sys[:,n] = y_sys[:,n] + 8.0 * noise[:,n]
        # system state equation
        if n < u.n_cols - 1:
            x_sys[:,n+1] = Ad_sys * x_sys[:,n] + Bd_sys * u[:,n]

        # observer output equation (kalman)
        y_estk[:,n] = Cd_est * x_estk[:,n] + Dd_est * u[:,n]
        # observer state equation (kalman)
        if n < u.n_cols - 1:
            x_estk[:,n+1] = Ad_est * x_estk[:,n] + Bd_est * u[:,n]
            # correction factor (stop correcting at time equals t >= predict_from)
            if t[0,n] < predict_from:
                y_errk = y_sys[:,n] - y_estk[:,n]
                x_estk[:,n+1] = x_estk[:,n+1] + Kd * y_errk
        
        # filter output equation (alternative)
        y_estf[:,n] = Cd_est * x_estf[:,n] + Dd_est * u[:,n]
        # filter state equation (alternative)
        if n < u.n_cols - 1:
            x_flt[:,n+1] = Ad_flt * x_flt[:,n]
            x_estf[:,n+1] = Ad_est * x_estf[:,n] + Bd_est * u[:,n] + (Bd_est*Cd_flt) * x_flt[:,n]
            # correction factor (stop correcting at time equals t >= predict_from)
            if t[0,n] < predict_from:
                y_errf = y_sys[:,n] - y_estf[:,n]
                x_flt[:,n+1] = x_flt[:,n+1] + Bd_flt * y_errf
                x_estf[:,n+1] = x_estf[:,n+1] + (Bd_est*Dd_flt) * y_errf

    # show simulation results
    sim_plot(t, x_sys, y_sys, x_estk, y_estk, x_estf, y_estf, predict_from)

In the `ipywidget` below, the only variable to tune for the alternative filter is `tau` ($\tau$), the lower the value, the lower the bandwidth (smoother but more phase delay), and the higher the value the higher the bandwidth (better tracking but less noise attenuation).

In [ ]:
interact(
    sim_filter_prediction, 
    q1 = (0.1, 1.0, 0.1),
    q2 = (0.1, 1.0, 0.1),
    r = (0.1, 1.0, 0.1),
    dist_k_factor = (0.1, 2, 0.1),
    dist_ramp_output = False,
    noise_output = True,
    predict_from = (10, 180, 10),
    tau = (0.05, 1, 0.05)
)

Note that with the alternative filter we have managed to reduce the number of tuning parameters to just one; $\tau$. And that it has a straightforward relation to a common filter specification; the bandwidth. Also note that for the parametric disturbance ($\hat{k} = k_{factor} \cdot k$), the alternative filter is able to produce much better output predictions. And for the case of the ramp disturbance, the alternative's filter predictions improve as the bandwidth ($\tau$) is increased, which agrees with what we would expect from control theory.

## Q & A

* This example is simple because it is a SISO system (Single Input Single Output), what if I have a MIMO system (Multiple Input/Output)?

The loopshaping problem has been solved for MIMO systems, actually it is quite auotmated now a days with tools like [Matlab's `loopsyn`](https://www.mathworks.com/help/robust/ref/dynamicsystem.loopsyn.html) (sadly not in Python at the time of writing). You just specify the open-loop desired dynamics and the rountine does the rest.

* This looks cool but does it work for real life applications? Given the improved predictions, could I use it for control (e.g. with [Model Predictive Control](https://en.wikipedia.org/wiki/Model_predictive_control), aka MPC)?

Yes and yes, in [this paper](https://folk.ntnu.no/skoge/prost/proceedings/ifac2014/media/files/2032.pdf), this very technique was used in a real drone to estimate and predict the states of an MPC controller.

* Does this mean that if my measurements contain some undesired frquency, I could just throw in a notch filter to $F(s)$ and that frequency would dissapear from the state estimates?

Yes, this is shown in [this thesis](https://research.tue.nl/nl/studentTheses/disturbance-model-and-observer-tuning-method-for-model-predictive) where this filter was first introduced.